# Distributed DeepRacer RL training with SageMaker and RoboMaker: Enhanced version for AIDO-3 NeurIPS DeepRacer challenge

---
## Introduction

This notebook is an enhanced version of [AWS DeepRacer](https://console.aws.amazon.com/deepracer/home#welcome), for AIDO-3 NeurIPS DeepRacer challenge. The notebook is an expansion of the original [Amazon SageMaker notebook](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/reinforcement_learning/rl_deepracer_robomaker_coach_gazebo) provides additional instructions to customize over the Amazon SageMaker training/evaluation jobs, AWS RoboMaker simulation assets, and RL algorithm tuning.


---
## How it works?  


The reinforcement learning agent (i.e. our autonomous car) learns to drive by interacting with its environment, e.g., the track, by taking an action in a given state to maximize the expected reward. The agent learns the optimal plan of actions in training by trial-and-error through repeated episodes.  
  
The figure above shows an example of distributed RL training across SageMaker and two RoboMaker simulation envrionments that perform the **rollouts** - execute a fixed number of episodes using the current model or policy. The rollouts collect agent experiences (state-transition tuples) and share this data with SageMaker for training. SageMaker updates the model policy which is then used to execute the next sequence of rollouts. This training loop continues until the model converges, i.e. the car learns to drive and stops going off-track. More formally, we can define the problem in terms of the following:  

1. **Objective**: Learn to drive autonomously by staying close to the center of the track.
2. **Environment**: A 3D driving simulator hosted on AWS RoboMaker.
3. **State**: The driving POV image captured by the car's head camera, as shown in the illustration above.
4. **Action**: Six discrete steering wheel positions at different angles (configurable)
5. **Reward**: Positive reward for staying close to the center line; High penalty for going off-track. This is configurable and can be made more complex (for e.g. steering penalty can be added).


## RoboMaker Simulation Assets

In this example, we use AWS RoboMaker, a cloud robotics service, to simulate the behavior of our robocar in a realistic simulation environment with a physics-engine and rendering for vision-based sensor input. 

We provide a pre-built simulation environment in `build/output.tar.gz` and Gazebo simulation assets for customizing your world environment in `build/simapp` folder. 

Download the pre-built simulation environment for customizing simulation assets and extract to the notebook folder.

In [1]:
# download
!aws s3 cp "s3://neurips-aido3-awsdeepracer/build.tar.gz" .
# untar
!tar -xzvf build.tar.gz

### How to add new tracks?
Raching tracks with various shapes and textures are included among the Gazebo assets `build/simapp/bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment`. To add a new track, add the following files:

- `worlds/myworld.world` which points to the track models, 
- `models/mytrack/mytrack.sdf` and `models/mytrack/model.config` which point to the meshes, 
- `meshes/mytrack/textures` and `meshes/mytrack/mytrack.dae` which contain the textures and meshes. For the already existing tracks, we provide Blender files for modifications as needed.
- `routes/mytrack.npy` which includes the waypoints data.

### How to re-bundle your simulation application?
After making changes to the simulation application assets, re-bundle it using the Python file sim_app_bundler.py. We will upload the tar.gz file to the AWS RoboMaker arn later in the notebook.

The compression may take longer depending on the instance type of your Amazon SageMaker notebook.

In [2]:
#Only when you have changes to SimApp
!python3 sim_app_bundler.py --tar

Bundling the SimApp output.tar.gz
Removing previously build build/output.tar.gz
Creating tar of build/simapp/bundle
Removing the previously created tar files
============ Archiving completed. Available at build/Output.tar.gz============ 



## Prequisites

### Imports

To get started, we'll import the Python libraries we need, set up the environment with a few prerequisites for permissions and configurations.

You can run this notebook from your local machine or from a SageMaker notebook instance. In both of these scenarios, you can run the following to launch a training job on SageMaker and a simulation job on RoboMaker.

In [3]:
import boto3
import sagemaker
import sys
import os
import re
import numpy as np
import subprocess
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from time import gmtime, strftime
import time
from IPython.display import Markdown
from markdown_helper import *

### Initializing basic parameters

In [4]:
# Select the instance type
#instance_type = "ml.c4.2xlarge"
#instance_type = "ml.c5.4xlarge"
instance_type = "ml.p2.xlarge"

# Starting SageMaker session
sage_session = sagemaker.session.Session()

# Create unique job name.
job_name_prefix = 'deepracer-notebook'

# Duration of job in seconds (1 hours)
job_duration_in_seconds = 3600 * 1

# AWS Region
aws_region = sage_session.boto_region_name
if aws_region not in ["us-west-2", "us-east-1", "eu-west-1"]:
    raise Exception("This notebook uses RoboMaker which is available only in US East (N. Virginia),"
                    "US West (Oregon) and EU (Ireland). Please switch to one of these regions.")

### Setup S3 bucket
Set up the linkage and authentication to the S3 bucket that we want to use for checkpoint and metadata.

In [5]:
# S3 bucket
s3_bucket = sage_session.default_bucket()

# SDK appends the job name and output folder
s3_output_path = 's3://{}/'.format(s3_bucket)

# Ensure that the S3 prefix contains the keyword 'sagemaker'
s3_prefix = job_name_prefix + "-sagemaker-" + strftime("%y%m%d-%H%M%S", gmtime())

# Get the AWS account id of this account
sts = boto3.client("sts")
account_id = sts.get_caller_identity()['Account']

print("Using s3 bucket {}".format(s3_bucket))
print("Model checkpoints and other metadata will be stored at: \ns3://{}/{}".format(s3_bucket, s3_prefix))

Using s3 bucket sagemaker-us-west-2-828753602420
Model checkpoints and other metadata will be stored at: 
s3://sagemaker-us-west-2-828753602420/deepracer-notebook-sagemaker-191111-193206


### Create an IAM role
Either get the execution role when running from a SageMaker notebook `role = sagemaker.get_execution_role()` or, when running from local machine, use utils method `role = get_execution_role('role_name')` to create an execution role.

In [6]:
try:
    sagemaker_role = sagemaker.get_execution_role()
except:
    sagemaker_role = get_execution_role('sagemaker')

print("Using Sagemaker IAM role arn: \n{}".format(sagemaker_role))

Using Sagemaker IAM role arn: 
arn:aws:iam::828753602420:role/service-role/AmazonSageMaker-ExecutionRole-20190604T142184


> Please note that this notebook cannot be run in `SageMaker local mode` as the simulator is based on AWS RoboMaker service.

### Permission setup for invoking AWS RoboMaker from this notebook
In order to enable this notebook to be able to execute AWS RoboMaker jobs, we need to add one trust relationship to the default execution role of this notebook.

In [7]:
display(Markdown(generate_help_for_robomaker_trust_relationship(sagemaker_role)))

1. Go to IAM console to edit current SageMaker role: [AmazonSageMaker-ExecutionRole-20190604T142184](https://console.aws.amazon.com/iam/home#/roles/AmazonSageMaker-ExecutionRole-20190604T142184).
2. Next, go to the `Trust relationships tab` and click on `Edit Trust Relationship.` 
3. Replace the JSON blob with the following:
```json
            {
              "Version": "2012-10-17",
              "Statement": [
                {
                  "Effect": "Allow",
                  "Principal": {
                    "Service": [
                      "sagemaker.amazonaws.com",
                      "robomaker.amazonaws.com"
                    ]
                  },
                  "Action": "sts:AssumeRole"
                }
              ]
            }```
4. Once this is complete, click on Update Trust Policy and you are done.

### Permission setup for Sagemaker to S3 bucket

The sagemaker writes the Redis IP address, models to the S3 bucket. This requires PutObject permission on the bucket. Make sure the sagemaker role you are using as this permissions.

In [8]:
display(Markdown(generate_s3_write_permission_for_sagemaker_role(sagemaker_role)))

1. Go to IAM console to edit current SageMaker role: [AmazonSageMaker-ExecutionRole-20190604T142184](https://console.aws.amazon.com/iam/home#/roles/AmazonSageMaker-ExecutionRole-20190604T142184).
2. Next, go to the `Permissions tab` and click on `Attach Policy.` 
3. Search and select `AmazonKinesisVideoStreamsFullAccess` policy


### Permission setup for Sagemaker to create KinesisVideoStreams

The sagemaker notebook has to create a kinesis video streamer. You can observer the car making epsiodes in the kinesis video streamer.

In [9]:
display(Markdown(generate_kinesis_create_permission_for_sagemaker_role(sagemaker_role)))

1. Go to IAM console to edit current SageMaker role: [AmazonSageMaker-ExecutionRole-20190604T142184](https://console.aws.amazon.com/iam/home#/roles/AmazonSageMaker-ExecutionRole-20190604T142184).
2. Next, go to the `Permissions tab` and click on `Attach Policy.` 
3. Search and select `AmazonS3FullAccess` policy


### Build and push docker image

The file ./Dockerfile contains all the packages that are installed into the docker. Instead of using the default sagemaker container. We will be using this docker container.

In [10]:
#!docker rm -f $(docker ps -a -q);
#!docker rmi -f $(docker images -q);

In [11]:
%%time
from copy_to_sagemaker_container import get_sagemaker_docker, copy_to_sagemaker_container, get_custom_image_name
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
# repo name
repository_short_name = job_name_prefix + "-%s" % cpu_or_gpu
custom_image_name = get_custom_image_name(repository_short_name)

try:
    print("Copying files from your notebook to existing sagemaker container")
    sagemaker_docker_id = get_sagemaker_docker(repository_short_name)
    copy_to_sagemaker_container(sagemaker_docker_id, repository_short_name)
except Exception as e:
    print("Creating sagemaker container")
    docker_build_args = {
        'CPU_OR_GPU': cpu_or_gpu, 
        'AWS_REGION': boto3.Session().region_name,
    }
    custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
    print("Using ECR image %s" % custom_image_name)
    


Copying files from your notebook to existing sagemaker container
docker images deepracer-notebook-gpu | sed -n 2,2p
Sagemaker docker id : 8a11a90a05a5
docker run -d -t 8a11a90a05a5
docker exec -d 888040893ed8609bae6cf59b965ac4385a80887bcd61c7800654d40dd9947b5e rm -rf /opt/amazon/markov
docker cp ./src/markov 888040893ed8609bae6cf59b965ac4385a80887bcd61c7800654d40dd9947b5e:/opt/amazon/markov
============ Copied Markov scripts to sagemaker docker ============ 
 
docker ps -l|sed -n 2,2p
docker commit 888040893ed8 deepracer-notebook-gpu
============ Commited all the changes to docker ============ 
 
CPU times: user 37.8 ms, sys: 27 ms, total: 64.8 ms
Wall time: 1.56 s


### Configure VPC

Since SageMaker and RoboMaker have to communicate with each other over the network, both of these services need to run in VPC mode. This can be done by supplying subnets and security groups to the job launching scripts.  
We will check if the deepracer-vpc stack is created and use it if present (This is present if the AWS Deepracer console is used atleast once to create a model). Else we will use the default VPC stack.

In [12]:
ec2 = boto3.client('ec2')

#
# Check if the user has Deepracer-VPC and use that if its present. This will have all permission.
# This VPC will be created when you have used the Deepracer console and created one model atleast
# If this is not present. Use the default VPC connnection
#
deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups']\
                             if group['GroupName'].startswith("deepracer-vpc")]
if(deepracer_security_groups):
    print("Using the DeepRacer VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] \
                     if "Tags" in vpc for val in vpc['Tags'] \
                     if val['Value'] == 'deepracer-vpc'][0]
    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc]
else:
    print("Using the default VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

    deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                                 if 'VpcId' in group and group["GroupName"] == "default" and group["VpcId"] == deepracer_vpc]

    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc and subnet['DefaultForAz']==True]

print("Using VPC:", deepracer_vpc)
print("Using security group:", deepracer_security_groups)
print("Using subnets:", deepracer_subnets)

Using the default VPC stacks
Using VPC: vpc-b418facc
Using security group: ['sg-fca8088c']
Using subnets: ['subnet-57a5de2e', 'subnet-67cc983d', 'subnet-b63155fd']


### Create Route Table
A SageMaker job running in VPC mode cannot access S3 resourcs. So, we need to create a VPC S3 endpoint to allow S3 access from SageMaker container. To learn more about the VPC mode, please visit [this link.](https://docs.aws.amazon.com/sagemaker/latest/dg/train-vpc.html)

In [13]:
#TODO: Explain to customer what CREATE_ROUTE_TABLE is doing
CREATE_ROUTE_TABLE = True

def create_vpc_endpoint_table():
    print("Creating ")
    try:
        route_tables = [route_table["RouteTableId"] for route_table in ec2.describe_route_tables()['RouteTables']\
                        if route_table['VpcId'] == deepracer_vpc]
    except Exception as e:
        if "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(sagemaker_role)))
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, deepracer_vpc)))
        raise e

    print("Trying to attach S3 endpoints to the following route tables:", route_tables)
    
    if not route_tables:
        raise Exception(("No route tables were found. Please follow the VPC S3 endpoint creation "
                         "guide by clicking the above link."))
    try:
        ec2.create_vpc_endpoint(DryRun=False,
                                VpcEndpointType="Gateway",
                                VpcId=deepracer_vpc,
                                ServiceName="com.amazonaws.{}.s3".format(aws_region),
                                RouteTableIds=route_tables)
        print("S3 endpoint created successfully!")
    except Exception as e:
        if "RouteAlreadyExists" in str(e):
            print("S3 endpoint already exists.")
        elif "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
            raise e
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, default_vpc)))
            raise e

if CREATE_ROUTE_TABLE:
    create_vpc_endpoint_table()

Creating 
Trying to attach S3 endpoints to the following route tables: ['rtb-0d549376']
S3 endpoint already exists.


## How to customize?

![How to customize](./SageMaker_RoboMaker.png)

We use horizontal scaling where the neural network model files are synchronized between the Amazon Sagemaker training job and AWS RoboMaker simulation workers. Model sync behavior is coded in src/markov/training_worker.py 

```
if graph_manager.agent_params.algorithm.distributed_coach_synchronization_type == 
            DistributedCoachSynchronizationType.SYNC:
    graph_manager.save_checkpoint()
else:
    graph_manager.occasionally_save_checkpoint()
```
To enable sync models to the s3 bucket for multiple rollouts, set the parameter in src/markov/presets/preset.py

```
agent_params.algorithm.distributed_coach_synchronization_type = DistributedCoachSynchronizationType.SYNC
```

In the following, we demonstrate how to customize the key components of the training agent such as reward and neural network architecture.

### Configure the environment file with custom simulation variables

We use an environment file, `src/markov/environment/deepracer_racetrack_env.py`, which contains "step" and "reset" functions and ability to exchange messages with the Gazebo based AWS RoboMaker simulator. This environment file is shared between Amazon Sagemaker and AWS RoboMaker jobs. The environment variable - `NODE_TYPE` defines which node the code is running on. So, the expressions that have `rospy` dependencies are executed on RoboMaker only. 

#### How to add noise to observations?
You can add noise to robocar camera observations by using OpenCV or other image editting packages available in Python. Note that these libraries need to be located in or copied to `build/simapp/bundle/usr/local/lib/python3.5/dist-packages` for the AWS RoboMaker to import them.

As an example, we provide a modified environment `src/markov/environment/deepracer_racetrack_env_cv2.py`, to use `cv2` and add Gaussian noise to robocar observations in `def set_next_state():` (see Lines ~241-254)

#### How to add noise to actions, i.e., steering and speed, for robustness?
Adding noise to your actions also increases the robustness of your model to steady-state or tracking errors of the robocar controllers for steering and speed in the real world. Since we use discrete action, we need to add noise to their associated mappings in `class DeepRacerRacetrackCustomActionSpaceEnv(DeepRacerRacetrackEnv):` (see Lines ~575-579)


In [14]:
# replace the environment file with your modified version
!cp src/markov/environments/deepracer_racetrack_env_cv2.py src/markov/environments/deepracer_racetrack_env.py

In [15]:
# Uncomment the pygmentize code lines to see the code

# Environmental File
#!pygmentize src/markov/environments/deepracer_racetrack_env.py

### Configure the neural network architecture and RL algorithm hyperparameters

To change the neural network architecture, edit the preset file in `src/markov/presets/`. We provide two example preset files:
- The default neural network architecture, `src/markov/presets/default.py`, has an input embedder with a 3 layer Convolutional Neural Network (CNN).
- The attention neural network architecture, `src/markov/presets/preset_attention_layer.py`, provides an example on how to use custom layers.

To change the RL agent algorithm, refer to the DQN example for AWS DeepRacer. There are multiple files that need to be editted including the preset file in `src/markov/presets/`. For this example, we use Clipped PPO. To edit the hyperparameters of the Clipped PPO RL agent, edit the preset file in `src/markov/presets/`. The configurable hyperparameters include learning_rate, neural network structure, batch_size, discount factor. 

In [16]:
# Preset File
#!pygmentize src/markov/presets/default.py
#!pygmentize src/markov/presets/preset_attention_layer.py

### Configure the reward function

To customize reward functions, modify `reward_function` in `src/markov/rewards/`. Note that the parameters exposed to the reward function are coded in the environment file. To create new variables for the reward function, edit the environment file, `src/markov/environment/deepracer_racetrack_env.py`.


In [17]:
# Reward function
#!pygmentize src/markov/rewards/default.py

### Configure the action space

Action space and steering angles can be changed by modifying `src/markov/actions/.json` file. The default action space for our RL agent is discrete, therefore, the number of actions correspond to the number of output nodes of the policy network.


In [18]:
# Action space
#!pygmentize src/markov/actions/model_metadata_10_state.json

### Copy custom files to S3 bucket so that sagemaker & robomaker can pick it up

In [19]:
s3_location = "s3://%s/%s" % (s3_bucket, s3_prefix)
print(s3_location)

# Clean up the previously uploaded files
!aws s3 rm --recursive {s3_location}

# Make any changes to the environment and preset files below and upload these files
!aws s3 cp src/markov/environments/deepracer_racetrack_env.py {s3_location}/environments/deepracer_racetrack_env.py

!aws s3 cp src/markov/rewards/default.py {s3_location}/rewards/reward_function.py

!aws s3 cp src/markov/actions/model_metadata_10_state.json {s3_location}/model_metadata.json

!aws s3 cp src/markov/presets/default.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_attention_layer.py {s3_location}/presets/preset.py

s3://sagemaker-us-west-2-828753602420/deepracer-notebook-sagemaker-191111-193206
upload: src/markov/environments/deepracer_racetrack_env.py to s3://sagemaker-us-west-2-828753602420/deepracer-notebook-sagemaker-191111-193206/environments/deepracer_racetrack_env.py
upload: src/markov/rewards/default.py to s3://sagemaker-us-west-2-828753602420/deepracer-notebook-sagemaker-191111-193206/rewards/reward_function.py
upload: src/markov/actions/model_metadata_10_state.json to s3://sagemaker-us-west-2-828753602420/deepracer-notebook-sagemaker-191111-193206/model_metadata.json
upload: src/markov/presets/default.py to s3://sagemaker-us-west-2-828753602420/deepracer-notebook-sagemaker-191111-193206/presets/preset.py


### Train the RL model using the Python SDK Script mode

Next, we define the following algorithm metrics that we want to capture from cloudwatch logs to monitor the training progress. These are algorithm specific parameters and might change for different algorithm. We use [Clipped PPO](https://coach.nervanasys.com/algorithms/policy_optimization/cppo/index.html) for this example.

In [20]:
metric_definitions = [
    # Training> Name=main_level/agent, Worker=0, Episode=19, Total reward=-102.88, Steps=19019, Training iteration=1
    {'Name': 'reward-training',
     'Regex': '^Training>.*Total reward=(.*?),'},
    
    # Policy training> Surrogate loss=-0.32664725184440613, KL divergence=7.255815035023261e-06, Entropy=2.83156156539917, training epoch=0, learning_rate=0.00025
    {'Name': 'ppo-surrogate-loss',
     'Regex': '^Policy training>.*Surrogate loss=(.*?),'},
     {'Name': 'ppo-entropy',
     'Regex': '^Policy training>.*Entropy=(.*?),'},
   
    # Testing> Name=main_level/agent, Worker=0, Episode=19, Total reward=1359.12, Steps=20015, Training iteration=2
    {'Name': 'reward-testing',
     'Regex': '^Testing>.*Total reward=(.*?),'},
]

We use the RLEstimator for training RL jobs.

1. Specify the source directory which has the environment file, preset and training code.
2. Specify the entry point as the training code
3. Specify the choice of RL toolkit and framework. This automatically resolves to the ECR path for the RL Container.
4. Define the training parameters such as the instance count, instance type, job name, s3_bucket and s3_prefix for storing model checkpoints and metadata. **Only 1 training instance is supported for now.**
4. Set the RLCOACH_PRESET as "deepracer" for this example.
5. Define the metrics definitions that you are interested in capturing in your logs. These can also be visualized in CloudWatch and SageMaker Notebooks.

In [21]:
estimator = RLEstimator(entry_point="training_worker.py",
                        source_dir='src',
                        image_name=custom_image_name,
                        dependencies=["common/"],
                        role=sagemaker_role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        train_max_run=job_duration_in_seconds,
                        hyperparameters={
                            "s3_bucket": s3_bucket,
                            "s3_prefix": s3_prefix,
                            "aws_region": aws_region,
                            "preset_s3_key": "%s/presets/preset.py"% s3_prefix,
                            "model_metadata_s3_key": "%s/model_metadata.json" % s3_prefix,
                            "environment_s3_key": "%s/environments/deepracer_racetrack_env.py" % s3_prefix,
                        },
                        subnets=deepracer_subnets,
                        security_group_ids=deepracer_security_groups,
                    )

estimator.fit(wait=False)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

Training job: deepracer-notebook-2019-11-11-19-32-12-595


### Create the Kinesis video stream (optional)

In [22]:
kvs_stream_name = "dr-kvs-{}".format(job_name)

!aws --region {aws_region} kinesisvideo create-stream --stream-name {kvs_stream_name} --media-type video/h264 --data-retention-in-hours 24
print ("Created kinesis video stream {}".format(kvs_stream_name))

{
    "StreamARN": "arn:aws:kinesisvideo:us-west-2:828753602420:stream/dr-kvs-deepracer-notebook-2019-11-11-19-32-12-595/1573500733492"
}
Created kinesis video stream dr-kvs-deepracer-notebook-2019-11-11-19-32-12-595


### Start the Robomaker job

In [23]:
robomaker = boto3.client("robomaker")

### Create Simulation Application

In [24]:
robomaker_s3_key = 'robomaker/simulation_ws.tar.gz'
robomaker_source = {'s3Bucket': s3_bucket,
                    's3Key': robomaker_s3_key,
                    'architecture': "X86_64"}
simulation_software_suite={'name': 'Gazebo',
                           'version': '7'}
robot_software_suite={'name': 'ROS',
                      'version': 'Kinetic'}
rendering_engine={'name': 'OGRE',
                  'version': '1.x'}

### Upload your customized simulation application to your s3 bucket

The AWS DeepRacer bundle to be used by the AWS RoboMaker service is under `build/output.tar.gz`. Next, we need to upload the bundle to our S3 bucket and create an AWS RoboMaker Simulation Application.

In [25]:
# Use your custom simApp
!aws s3 cp ./build/output.tar.gz s3://{s3_bucket}/{robomaker_s3_key}

upload: build/output.tar.gz to s3://sagemaker-us-west-2-828753602420/robomaker/simulation_ws.tar.gz


### Create arn for the AWS RoboMaker simulation application

In [26]:
# create arn for AWS RoboMaker
app_name = "deepracer-notebook-application" + strftime("%y%m%d-%H%M%S", gmtime())

print(app_name)
try:
    response = robomaker.create_simulation_application(name=app_name,
                                                       sources=[robomaker_source],
                                                       simulationSoftwareSuite=simulation_software_suite,
                                                       robotSoftwareSuite=robot_software_suite,
                                                       renderingEngine=rendering_engine)
    simulation_app_arn = response["arn"]
    print("Created a new simulation app with ARN:", simulation_app_arn)
except Exception as e:
    if "AccessDeniedException" in str(e):
        display(Markdown(generate_help_for_robomaker_all_permissions(role)))
        raise e
    else:
        raise e

deepracer-notebook-application191111-193223
Created a new simulation app with ARN: arn:aws:robomaker:us-west-2:828753602420:simulation-application/deepracer-notebook-application191111-193223/1573500743944


### Launch the Simulation job on RoboMaker

We create [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome) Simulation Jobs that simulates the environment and shares this data with SageMaker for training. 

In [27]:
# enter the number of workers
num_simulation_workers = 1

# select the world name
# check AWS DeepRacer console or assets in build folder for additional tracks
available_train_tracks = ["reinvent_base", # 0
                    "AWS_track", # 1
                    "Tokyo_Training_track", #2
                    "Virtual_May19_Train_track", #3 (london)
                    "reInvent2018_36inch"] #4

# choose the world
WORLD_NAME = available_train_tracks[0]

print('WORLD NAME = ', WORLD_NAME)

WORLD NAME =  reinvent_base


In [28]:
# environment variables
envriron_vars = {
    "WORLD_NAME": WORLD_NAME,
    "KINESIS_VIDEO_STREAM_NAME": kvs_stream_name,
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "TRAINING_JOB_ARN": job_name,
    "APP_REGION": aws_region,
    "METRIC_NAME": "TrainingRewardScore",
    "METRIC_NAMESPACE": "AWSDeepRacer",
    "REWARD_FILE_S3_KEY": "%s/rewards/reward_function.py" % s3_prefix,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model_metadata.json" % s3_prefix,
    "METRICS_S3_BUCKET": s3_bucket,
    "METRICS_S3_OBJECT_KEY": s3_bucket + "/training_metrics.json",
    "TARGET_REWARD_SCORE": "None",
    "NUMBER_OF_EPISODES": "0",
    "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID": account_id
}

simulation_application = {"application":simulation_app_arn,
                          "launchConfig": {"packageName": "deepracer_simulation_environment",
                                           "launchFile": "distributed_training.launch",
                                           "environmentVariables": envriron_vars}
                         }


vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

client_request_token = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

responses = []
#  failureBehavior="Fail" or  failureBehavior="Continue" if 2 or more workers
for job_no in range(num_simulation_workers):
    response =  robomaker.create_simulation_job(iamRole=sagemaker_role,
                                            clientRequestToken=client_request_token,
                                            maxJobDurationInSeconds=job_duration_in_seconds,
                                            failureBehavior="Fail",
                                            simulationApplications=[simulation_application],
                                            vpcConfig=vpcConfig
                                            )
    responses.append(response)

print("Created the following jobs:")
job_arns = [response["arn"] for response in responses]
for response in responses:
    print("Job ARN", response["arn"]) 

Created the following jobs:
Job ARN arn:aws:robomaker:us-west-2:828753602420:simulation-job/sim-tlx5szzdd48g


### Visualizing the simulations in RoboMaker
You can visit the RoboMaker console to visualize the simulations or run the following cell to generate the hyperlinks.

In [29]:
display(Markdown(generate_robomaker_links(job_arns, aws_region)))

> Click on the following links for visualization of simulation jobs on RoboMaker Console
- [Simulation 1](https://us-west-2.console.aws.amazon.com/robomaker/home?region=us-west-2#simulationJobs/sim-tlx5szzdd48g)  

You can click on Gazebo after you open the above link to start the simulator.

### Evaluation - ReInvent Track

For evaluation, refer to the companion notebooks for AIDO-3 NeurIPS DeepRacer challenge.

### Clean up RoboMaker and SageMaker training job

Execute the cells below if you want to kill RoboMaker and SageMaker job.

In [30]:
# # Cancelling robomaker job
# for job_arn in job_arns:
#     robomaker.cancel_simulation_job(job=job_arn)

# # Stopping sagemaker training job
# sage_session.sagemaker_client.stop_training_job(TrainingJobName=estimator._current_job_name)

### Clean Up Simulation Application Resource

In [31]:
# robomaker.delete_simulation_application(application=simulation_app_arn)

### Clean your S3 bucket (Uncomment the awscli commands if you want to do it)

In [32]:
## Uncomment if you only want to clean the s3 bucket
# sagemaker_s3_folder = "s3://{}/{}".format(s3_bucket, s3_prefix)
# !aws s3 rm --recursive {sagemaker_s3_folder}

# robomaker_s3_folder = "s3://{}/{}".format(s3_bucket, job_name)
# !aws s3 rm --recursive {robomaker_s3_folder}

# robomaker_sim_app = "s3://{}/{}".format(s3_bucket, 'robomaker')
# !aws s3 rm --recursive {robomaker_sim_app}

# model_output = "s3://{}/{}".format(s3_bucket, s3_bucket)
# !aws s3 rm --recursive {model_output}

### Clean the docker images
Remove this only when you want to completely remove the docker or clean up the space of the sagemaker instance

In [33]:
# !docker rmi -f $(docker images -q)